# Sea ice decline and Aitkensize particle relation

#### Tuuli Lehmusjärvi (tuuli.lehmusjarvi@helsinki.fi)
2 Nov, 2019


NeGI course 2019 - Climate in high latitudes: eScience for linking Arctic measurements and modeling

Group assistant: Lisa Beck

# Introduction

Aerosols have a direct and indirect effect on the radiative balance: direct effects are scattering, reflection and absorbtion of short wave radiation. THe indirect effect is occuring  when aerosol particles act as Cloud Condensation Nuclei (CCN) and affect the clound's properties such as life time and reflectivity. Currently atmospheric aerosols cause the larges uncertainties in global radiative forcing predictions and this is the biggest in the Arctic regions (Dall'Osto 2018).   

New particle formation plays a big role in the CCN formation. In the Arctic summer new particle formation is the biggest and even though it occurs regionaly it affects worldwide aerosol number concentrations. New particles in the Arctic are mainly formed due to emission of biogenic sulphur gases(Dall'Osto 2018). 

In the course my task was to look atobservation data of particle size distribution from Zeppelin station at Ny-Ålesund and compare that to reanalyzed sea ice extend data. The aim was to see if sea ice loss is affecting the particle size distribution. Dall’Osto et al. 2018 claim that declining sea ice and therefore increased exposure of open water is increasing new particle formation in the Arctic. They were concentrating on observational data collected at Villum, North East Greenland. The idea is to study this effect in Svalbard. The reanalyzed data showed the ice extent in the whole Arctic sea from 1979-2012, but I wanted to concentrate on the area in the west of Svalbard (Fram Strait), which affects most to the measurements done at Zeppelin since it is located at the west coast. From the particle size distribution data from 2000-2016 I wanted to select the time periods when the wind was blowing from the west or north-west. With this filter I hoped to better see the link between the sea ice extent in the west of Svalbard and the particle concentration. However this turned out to be not as succesful as expected way and therefore to using all the observed and modelled data. 

The observational dataset of particle size distribution was from NILU and I got two datasets from EBAS (years 2000-2007 and 2008-2009) and the third dataset (2010-2016) from the NeGI server.(?)
The reanalyzed dataset I used was ERA-Interim from ECMWF for the years 1979-2012.

In this notebook in methods section ([3](#Methods)) I will show how I analysed the both modelled and observed data and how I will comperare those two. In the results and discussion section ([4](#Results-and-discussion)) I will show my main finding and analysis from the data and parameters shown in methods.

# Methods

Reading, analyzing and plottinf of the observational data is presented in  [Section 3.1](#Observational-data). [Section 3.2](#Reanalyzed-model-data) will show the same thing but with model data. All the code is commented to make it easier to understand. 

In [ ]:

# Import all needed packages
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pyaerocom as pya
import glob
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from IPython.display import Image
from tuuli_functions import size_dist_to_xarray
from tuuli_functions import lognorm_to_concentration
from tuuli_functions import load_seaice_xarray


## Observational data

The observational data used was aerosol particle size distribution data from Zeppelin station in Ny-Ålesund Svalbard. It was fetched from EBAS database. I wanted to have as long as possible time series and I ended up taking data from 2000 until 2015. A single long time series was not available in EBAS, but two smaller ones: first from 2000 to 2007 and second from 2008 to 2009. The third dataset I found was from our NEGi course folder and it was from 2010 to 2016. So now I had three different datasets which needed to be read separatly due to different file formats. I also ended up using the last dataset only until end of 2012, because the model data I used also ended in that year. In addition, there were many major data gaps in 2014 and 2015 which could have made the analysis unsure.  The particle size range in the data was from 20-500nm. Since I wanted to look the Aitkensize particles, I selected size range of 20-50nm from all the three datasets.

I also planned to use wind direction data from Zeppelin station from the same years I had the particle size distributions datasets (2000-2012). Eventually, I decided to drop the wind data analysis since it didn't work  how I was planned. In the subsection ([3.1.4](#Wind-data-from-Zeppelin)) the reading and processing of the wind data will still be shown, and in results and discussion ([4](#Results-and-discussion)) I will elaborate on the reasons of disregarding the data.

### Reading and cleaning the different datasets

First, the datasets needed to be loaded into memory from the different files. After that all the flag values must be removed and new arrays created from the raw data. First the years 2000-2007:

In [ ]:

# Reading all the files in the firs part of the datasets (2000-2007)
# together into a new array with pyaerocom
arrs1 = []
for filepath in glob.glob('EBAS_FILES_2000_2007/*.nas'):
    filedata = pya.io.EbasNasaAmesFile(filepath)
    arrs1.append(size_dist_to_xarray(filedata))    

# Sorting the previously acquired array by time
arr1 = arrs1[0]
for array1 in arrs1[1:]:
    arr1 = xr.concat([arr1, array1], dim='time')


Next the years 2008-2009:

In [ ]:

# Reading all the files in the second part of the datasets (2008-2009)
# together into a new array with pyaerocom
arrs2 = []
for filepath2 in glob.glob('ebasfiles_2008_2009/*.nas'):
    filedata2 = pya.io.EbasNasaAmesFile(filepath2)
    arrs2.append(size_dist_to_xarray(filedata2, step=1))
    
# Sorting the previously acquired array by time
arr2 = arrs2[0]
for array2 in arrs2[1:]:
    arr2 = xr.concat([arr2, array2], dim='time')  
  

Last the years 2010-2015. Here the reading is done little bit differently since the format of the files was .csv and easier to read with Pandas builtins than the .nas files of the two previous datasets.

In [12]:

#Reading the last part of datasets (2010-2015)
datadir = '/home/2daa7756-2d5725-2d4dfb-2db0ff-2d5e0a6858a009/shared-ns1000k/inputs//Aerosol_sizedist_obs/'
filenam1 = datadir + 'Zeppelin_2010_hourly.csv'
filenam2 = datadir + 'Zeppelin_2011_hourly.csv'
filenam3 = datadir + 'Zeppelin_2012_hourly.csv'
filenam4 = datadir + 'Zeppelin_2013_hourly.csv'
filenam5 = datadir + 'Zeppelin_2015_hourly.csv'

flist=[filenam1, filenam2, filenam3, filenam4, filenam5]

# Creating a date parser
mydateparser = lambda x: pd.datetime.strptime(x, "%Y %m %d %H %M")

# Go through each file and append them together. 
# Date is split in five first columns, so we parse them together using 'mydateparser'.

datalist = []
for f in flist:
    datalist.append(pd.read_csv(f, parse_dates=[['0','0.1','0.2','0.3','0.4']],date_parser = mydateparser))
# Convert the new 'datalist' to Paandas dataframe    
data3 = pd.concat(datalist, axis=0)


To convert the first two datasets from raw data to readable arrays I used a function called 'size_dist_to_xarray': 

```python
def size_dist_to_xarray(loaded_ebas_file, step=2 ):
    filedata = loaded_ebas_file
    all_d = filedata.data[:,2:-1:step]
    time =  list(filedata.time_stamps)
    ds = xr.Dataset({'time':time})

    ds['sized'] = xr.DataArray(
        all_d, 
        dims={'time':ds['time'], 'd_index': np.arange(len(all_d[0,:]))}
    )
    return ds['sized']
```
This function will remove the flag values from the raw data and returns it in an xarray.

### Processing data

Now the data which is stored in xarrays should be converted to Pandas dataframes for easier joining. I wanted them all be in the same format. Some of the data in the 2008-2009 dataset was clearly faulty. In this section, I also show the removal of these erroneous data points. The data in the other data sets was usable as is.

The particle size distribution data doesn’t directly give the concentration of the particles. The instruments are measuring lognormat distributions so the unit of the 'concentrations' is actually:

$$\frac{dN_i}{d\log{D_p}}$$ 

Where $N_i$ is the concentration of the particles, and $D_p$ is the diameter of the particles. So when processing the data I need to calculate the real concentration from the measured data. For that I used a funtion called 'lognorm_to_concentration'.The function parameters are the name of the dataframe with the cleaned data, start and end date from the dataset and the names of the starting column and ending column. These columns define the size range of the particles that I want to study (20-50). Now we can integrate over $log10(D_p)$ to get the real concentration value; the return value of the function. 

```python
# Get data for diameters within 20-50nm
def lognorm_to_concentration(data, start_col, end_col, start_date, end_date):
    df_sizes = data.loc[start_col:end_col,start_date:end_date].T

# Integrate over log10(Dp) to get Ntot
    Ntot = pd.Series(np.trapz(df_sizes,
    x = np.log10(df_sizes.columns)),
    index=df_sizes.index.copy())
    
    return Ntot
```

Processing the first dataset:

In [ ]:

# Turn arr1 to Pandas-dataframe for easier handling
df1 = arr1.to_dataframe().unstack('d_index')
cols = [x[1] for x in df1.columns.values]
# Transpose the dataframe for the futher analysis
df1_turned = df1.T.loc['sized']

# Get data for 1 Mar 2000 - 31 Dec 2007 for diameters within 20-50nm
# Column names 1-5 represents diameters from 20-50nm 
Ntot_20_50_2000_2007 = lognorm_to_concentration(df1_turned,1,5,'2000-03-01 00:30:00','2007-12-31 19:29:59')

Processing the second dataset and removing incorrect data :

In [ ]:

# Turn arr1 to Pandas-dataframe for easier handling
df2 = arr2.to_dataframe().unstack('d_index')
cols2 = [y[1] for y in df2.columns.values]

# Masking the data to remove the overly large values which are not real

dd=df2.iloc[:,3:8] #First selecting right particle size range (20-50nm) to a new dataframe dd
mask = dd >10000 #Creating a mask of values over than 10000
dd[mask] = np.nan #Setting those values to NaN

# Plotting shows that in the end of 2009 there is weird peak
dd.plot(figsize=(14, 6))
plt.ylabel('Particle size distribution')
plt.savefig('particle_size_08_09_not_converted.png')

![title](particle_size_08_09_not_converted.png)

In [ ]:

# Plotting only the weird subset to see better
weird_subset = dd["2008-12-01":"2008-12-24"]
weird_subset.plot(figsize=(14, 6))
plt.ylabel('Particle size distribution')
plt.savefig('particle_size_weird_subset.png')



![title](particle_size_weird_subset.png)

In [ ]:

# Setting the time when the weird subset happened to NaN 
dd["2008-12-01":"2008-12-24"] = np.nan

# Transpose the dataframe for the futher analysis
dd_turned = dd.T.loc['sized']

# Get data for 1 Jan 2008 - 31 Dec 2009 for diameters within 20-50nm
# Column names 3-7 represents diameters from 20-50nm 
Ntot_20_50_2008_2009 = lognorm_to_concentration(dd_turned,3,7,'2008-01-01 00:30:00','2009-12-31 23:29:59')


Processing the third dataset:

In [ ]:

# When parsing the header is also affected so we give it a new name, 'date'
data3.rename(columns={'0_0.1_0.2_0.3_0.4':'date'}, inplace = True)
# Set indices of the rows to date
data3 = data3.set_index('date')
# Remove last column
data3.drop(labels='0.6', axis=1, inplace=True) 

# Change all the incorrect values (-999) to NaN
data3 = data3.replace(-999,np.nan)

# Check the names of the columns, now they are the same as the diameter of the particles 
data3.columns = [float(ii) for ii in data3.columns]

# Transpose the dataframe for the futher analysis
new_d=data3.T
# Get data for 1 Jan 2010 - 28 Aug 2013 for diameters within 20-50nm
# Column names 20.0-50.238represents diameters from 20-50nm 
Ntot_20_50_2010_2013 = lognorm_to_concentration(new_d,20.0,50.238,'2010-01-01 00:00:00','2012-12-31 08:00:00')


### Plotting data

When processing data, it’s a good practice to have a look a the data as a sanity check to make sure everything is fine. 

In [ ]:
# Take a look at the data as a sanity check
Ntot_20_50_2000_2007.plot(figsize=(12,6))
plt.ylabel('Particle concentration')
plt.savefig('sanitycheck_1.png')

![title](sanitycheck_1.png)

In [ ]:
# Take a look at the data as a sanity check
Ntot_20_50_2008_2009.plot(figsize=(12,6))
plt.ylabel('Particle concentration')
plt.savefig('sanitycheck_2.png')

![title](sanitycheck_2.png)

In [ ]:
# Take a look at the data as a sanity check
Ntot_20_50_2010_2013.plot(figsize=(12,6))
plt.ylabel('Particle concentration')
plt.savefig('sanitycheck_3.png')

![title](sanitycheck_3.png)

### Wind data from Zeppelin

Reading ad cleaning the wind data is done much in the same way as the particle data before. 

In [ ]:
# reading all the wind direction files together into a new array with pyaerocom
winddir = []
for filepath4 in glob.glob('ebas_winddir/*.nas'):
    filedata4 = pya.io.EbasNasaAmesFile(filepath4)
    winddir.append(filedata4.to_dataframe().wind_direction_deg)
    
# Sorting the previously acquired array by time
all_wind = winddir[0]
for data in winddir[1:]:
    all_wind = pd.concat([all_wind, data], axis=0)
all_wind.sort_index(inplace=True)    

To use the wind data as a mask to choose the correct days from the processed particle data, they need to have exactly the  same time frame. Here I created the filter mask to match the time of the particle data, and selected only the times when the wind came from between west and northwest. 

In [ ]:
# Taking hourly mean from the data.
# Wind data and concentration data needs to start at the same time step that they can be used together
all_wind_avg = all_wind.resample('H').mean()

# Creating a mask for the wind directions. 
# Selecting only those times (hours) when wind comes from west or northwest
filter_mask = all_wind_avg.between(260, 350) 
# Concentration starts on 1st of March 2000, so we need to make the wind data to start at the same time 
March_index = filter_mask.index > pd.to_datetime('2000-03-01 00:00:00')  
new_filter = filter_mask[March_index]

## Reanalyzed model data

### Reading the model data


# Results and discussion